In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.metrics as metrics
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import datetime
import math
import yfinance as yf
import xgboost as xg
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

In [ ]:
start_date = '2000-01-01'
end_date = '2023-02-05'
ticker = '^GSPC'
data = yf.download(ticker, start_date, end_date)
print(data.tail())

[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2023-01-30  4049.270020  4063.850098  4015.550049  4017.770020  4017.770020   
2023-01-31  4020.850098  4077.159912  4020.439941  4076.600098  4076.600098   
2023-02-01  4070.070068  4148.950195  4037.199951  4119.209961  4119.209961   
2023-02-02  4158.680176  4195.439941  4141.879883  4179.759766  4179.759766   
2023-02-03  4136.689941  4182.359863  4123.359863  4136.479980  4136.479980   

                Volume  
Date                    
2023-01-30  3802000000  
2023-01-31  4679320000  
2023-02-01  4856930000  
2023-02-02  5624360000  
2023-02-03  4694510000  


In [ ]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5810.000000,5810.000000,5810.000000,5810.000000,5810.000000,5.810000e+03
mean,1881.920683,1893.190926,1869.724736,1882.144273,1882.144273,3.301117e+09
std,972.577001,977.691103,967.047736,972.723195,972.723195,1.524059e+09
min,679.280029,695.270020,666.789978,676.530029,676.530029,3.560700e+08
25%,1184.245026,1191.389984,1176.972443,1184.092468,1184.092468,1.981010e+09
50%,1433.214966,1441.719971,1424.984985,1433.229980,1433.229980,3.406545e+09
75%,2382.589966,2391.074951,2369.127441,2382.820068,2382.820068,4.160128e+09
max,4804.509766,4818.620117,4780.040039,4796.560059,4796.560059,1.145623e+10


In [ ]:
df = pd.DataFrame(data)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000
...,...,...,...,...,...,...
2023-01-30,4049.270020,4063.850098,4015.550049,4017.770020,4017.770020,3802000000
2023-01-31,4020.850098,4077.159912,4020.439941,4076.600098,4076.600098,4679320000
2023-02-01,4070.070068,4148.950195,4037.199951,4119.209961,4119.209961,4856930000


In [ ]:
sma_window = 5
ema_span = 5
df['SMA_Open'] = df['Open'].rolling(window=sma_window).mean()
df['SMA_High'] = df['High'].rolling(window=sma_window).mean()
df['SMA_Low'] = df['Low'].rolling(window=sma_window).mean()
df['SMA_Close'] = df['Close'].rolling(window=sma_window).mean()
df['EMA_Open'] = df['Open'].ewm(span=ema_span, adjust=False).mean()
df['EMA_High'] = df['High'].ewm(span=ema_span, adjust=False).mean()
df['EMA_Low'] = df['Low'].ewm(span=ema_span, adjust=False).mean()
df['EMA_Close'] = df['Close'].ewm(span=ema_span, adjust=False).mean()

In [ ]:
rsi_window = 5
delta = df['Open'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=rsi_window).mean()
avg_loss = loss.rolling(window=rsi_window).mean()
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))
df['RSI_Open'] = rsi

In [ ]:
rsi_window = 5
delta = df['High'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=rsi_window).mean()
avg_loss = loss.rolling(window=rsi_window).mean()
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))
df['RSI_High'] = rsi

In [ ]:
rsi_window = 5
delta = df['Low'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=rsi_window).mean()
avg_loss = loss.rolling(window=rsi_window).mean()
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))
df['RSI_Low'] = rsi

In [ ]:
rsi_window = 5
delta = df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=rsi_window).mean()
avg_loss = loss.rolling(window=rsi_window).mean()
rs = avg_gain / avg_loss
rsi = 100 - (100 / (1 + rs))
df['RSI_Close'] = rsi

In [ ]:
ema_fast = df['Open'].ewm(span=12, adjust=False).mean()
ema_slow = df['Open'].ewm(span=26, adjust=False).mean()
macd = ema_fast - ema_slow
signal_line = macd.ewm(span=5, adjust=False).mean()
df['MACD_Open'] = macd
df['Signal_Line_Open'] = signal_line

In [ ]:
ema_fast = df['High'].ewm(span=12, adjust=False).mean()
ema_slow = df['High'].ewm(span=26, adjust=False).mean()
macd = ema_fast - ema_slow
signal_line = macd.ewm(span=5, adjust=False).mean()
df['MACD_High'] = macd
df['Signal_Line_High'] = signal_line

In [ ]:
ema_fast = df['Low'].ewm(span=12, adjust=False).mean()
ema_slow = df['Low'].ewm(span=26, adjust=False).mean()
macd = ema_fast - ema_slow
signal_line = macd.ewm(span=5, adjust=False).mean()
df['MACD_Low'] = macd
df['Signal_Line_Low'] = signal_line

In [ ]:
ema_fast = df['Close'].ewm(span=12, adjust=False).mean()
ema_slow = df['Close'].ewm(span=26, adjust=False).mean()
macd = ema_fast - ema_slow
signal_line = macd.ewm(span=5, adjust=False).mean()
df['MACD_Close'] = macd
df['Signal_Line_Close'] = signal_line

In [ ]:
change = df['Open'].diff()
volume_direction = pd.Series(0, index=df.index)
volume_direction[change > 0] = 1
volume_direction[change < 0] = -1
volume_direction_cumsum = volume_direction.cumsum()
df['OBV_Open'] = volume_direction_cumsum * df['Volume']

In [ ]:
change = df['High'].diff()
volume_direction = pd.Series(0, index=df.index)
volume_direction[change > 0] = 1
volume_direction[change < 0] = -1
volume_direction_cumsum = volume_direction.cumsum()
df['OBV_High'] = volume_direction_cumsum * df['Volume']

In [ ]:
change = df['Low'].diff()
volume_direction = pd.Series(0, index=df.index)
volume_direction[change > 0] = 1
volume_direction[change < 0] = -1
volume_direction_cumsum = volume_direction.cumsum()
df['OBV_Low'] = volume_direction_cumsum * df['Volume']

In [ ]:
change = df['Close'].diff()
volume_direction = pd.Series(0, index=df.index)
volume_direction[change > 0] = 1
volume_direction[change < 0] = -1
volume_direction_cumsum = volume_direction.cumsum()
df['OBV_Close'] = volume_direction_cumsum * df['Volume']

In [ ]:
n = 5
low_14, high_14 = df['Low'].rolling(window=n).min(), df['High'].rolling(window=n).max()
df['%K_Open'] = 100 * (df['Open'] - low_14) / (high_14 - low_14)
df['%D_Open'] = df['%K_Open'].rolling(window=5).mean()

In [ ]:
n = 5
low_14, high_14 = df['Low'].rolling(window=n).min(), df['High'].rolling(window=n).max()
df['%K_High'] = 100 * (df['High'] - low_14) / (high_14 - low_14)
df['%D_High'] = df['%K_High'].rolling(window=5).mean()

In [ ]:
n = 5
low_14, high_14 = df['Low'].rolling(window=n).min(), df['High'].rolling(window=n).max()
df['%K_Low'] = 100 * (df['Low'] - low_14) / (high_14 - low_14)
df['%D_Low'] = df['%K_Low'].rolling(window=5).mean()

In [ ]:
n = 5
low_14, high_14 = df['Low'].rolling(window=n).min(), df['High'].rolling(window=n).max()
df['%K_Close'] = 100 * (df['Close'] - low_14) / (high_14 - low_14)
df['%D_Close'] = df['%K_Close'].rolling(window=5).mean()

In [ ]:
period = 5
df['Open_Percentage'] = df['Open'].pct_change(periods=period) * 100
df['High_Percentage'] = df['High'].pct_change(periods=period) * 100
df['Low_Percentage'] = df['Low'].pct_change(periods=period) * 100
df['Volume_Percentage'] = df['Volume'].pct_change(periods=period) * 100
df['Close_Percentage'] = df['Close'].pct_change(periods=period) * 100

In [ ]:
df.shape

(5810, 43)

In [ ]:
df.head(5).mean()

Open                 1.425890e+03
High                 1.439972e+03
Low                  1.401260e+03
Close                1.420334e+03
Adj Close            1.420334e+03
Volume               1.068760e+09
SMA_Open             1.425890e+03
SMA_High             1.439972e+03
SMA_Low              1.401260e+03
SMA_Close            1.420334e+03
EMA_Open             1.445324e+03
EMA_High             1.455454e+03
EMA_Low              1.415602e+03
EMA_Close            1.432121e+03
RSI_Open             5.456153e+00
RSI_High             3.090830e+01
RSI_Low              2.752892e+01
RSI_Close            4.297393e+01
MACD_Open           -6.197786e+00
Signal_Line_Open    -3.105851e+00
MACD_High           -5.898844e+00
Signal_Line_High    -3.081971e+00
MACD_Low            -6.010517e+00
Signal_Line_Low     -3.270114e+00
MACD_Close          -6.162109e+00
Signal_Line_Close   -3.484693e+00
OBV_Open            -8.544600e+08
OBV_High            -1.781460e+09
OBV_Low             -8.544600e+08
OBV_Close     

In [ ]:
df.head(20)

,Open,High,Low,Close,Adj Close,Volume,SMA_Open,SMA_High,SMA_Low,SMA_Close,...,%D_High,%K_Low,%D_Low,%K_Close,%D_Close,Open_Percentage,High_Percentage,Low_Percentage,Volume_Percentage,Close_Percentage
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,1425.889990,1439.971997,1401.260010,1420.333984,...,NaN,22.976415,NaN,63.586474,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,1441.469971,1464.359985,1441.469971,1457.599976,1457.599976,1064800000,1420.333984,1437.243994,1401.882007,1420.809985,...,NaN,73.592487,NaN,92.201182,NaN,-1.890763,-0.922870,0.216217,14.273449,0.163549
2000-01-11,1457.599976,1458.660034,1434.420044,1438.560059,1438.560059,1014000000,1420.809985,1437.932007,1409.280005,1428.637988,...,NaN,65.459200,NaN,70.235409,NaN,0.163549,0.236395,2.647001,0.495540,2.796874
2000-01-12,1438.560059,1442.599976,1427.079956,1432.250000,1432.250000,974600000,1428.637988,1443.797998,1419.159985,1434.665991,...,NaN,48.408491,NaN,55.563270,NaN,2.796874,2.075326,3.585731,-10.216490,2.149618
2000-01-13,1432.250000,1454.199951,1432.250000,1449.680054,1449.680054,1030400000,1434.665991,1452.257983,1427.189990,1443.912012,...,82.185952,49.536409,51.994600,76.929231,71.703113,2.149618,2.995958,2.884134,-5.666941,3.294033


In [ ]:
df1 = df.dropna()

In [ ]:
df1

,Open,High,Low,Close,Adj Close,Volume,SMA_Open,SMA_High,SMA_Low,SMA_Close,...,%D_High,%K_Low,%D_Low,%K_Close,%D_Close,Open_Percentage,High_Percentage,Low_Percentage,Volume_Percentage,Close_Percentage
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-13,1432.250000,1454.199951,1432.250000,1449.680054,1449.680054,1030400000,1434.665991,1452.257983,1427.189990,1443.912012,...,82.185952,49.536409,51.994600,76.929231,71.703113,2.149618,2.995958,2.884134,-5.666941,3.294033
2000-01-14,1449.680054,1473.000000,1449.680054,1465.150024,1465.150024,1085900000,1443.912012,1458.563989,1436.980005,1448.648022,...,89.468658,49.216193,57.242556,82.905122,75.566843,3.294033,2.187352,3.494612,-11.369572,1.642771
2000-01-18,1465.150024,1465.150024,1451.300049,1455.140015,1455.140015,1056700000,1448.648022,1458.721997,1438.946021,1448.156030,...,86.049682,52.744054,53.072870,61.106341,69.347875,1.642771,0.053951,0.681948,-0.760706,-0.168768
2000-01-19,1455.140015,1461.390015,1448.680054,1455.900024,1455.900024,1087800000,1448.156030,1459.267993,1441.798022,1451.624023,...,82.308245,47.038495,49.388729,62.761413,67.853075,-0.168768,0.187157,0.994131,7.278107,1.205370
2000-01-20,1455.900024,1465.709961,1438.540039,1445.569946,1445.569946,1100700000,1451.624023,1463.889990,1444.090039,1454.288013,...,84.753009,15.435679,42.794166,32.686985,63.277818,1.205370,1.601968,0.803044,12.938641,0.930001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-30,4049.270020,4063.850098,4015.550049,4017.770020,4017.770020,3802000000,4024.704004,4052.620020,4003.278027,4036.385986,...,91.244976,45.807809,58.813463,47.337242,80.158024,1.788025,0.607530,1.105593,-3.629972,-0.050750
2023-01-31,4020.850098,4077.159912,4020.439941,4076.600098,4076.600098,4679320000,4028.526025,4063.268018,4009.408008,4048.316016,...,90.897382,49.176666,55.089586,87.867809,80.639839,0.477545,1.323088,0.768208,40.925121,1.484961
2023-02-01,4070.070068,4148.950195,4037.199951,4119.209961,4119.209961,4856930000,4045.998047,4089.148047,4027.035986,4068.914014,...,93.467455,17.624860,50.352867,78.077399,79.258516,2.193484,3.219269,2.231921,30.421695,2.564351


In [ ]:
df1.isnull().sum()

Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
Volume               0
SMA_Open             0
SMA_High             0
SMA_Low              0
SMA_Close            0
EMA_Open             0
EMA_High             0
EMA_Low              0
EMA_Close            0
RSI_Open             0
RSI_High             0
RSI_Low              0
RSI_Close            0
MACD_Open            0
Signal_Line_Open     0
MACD_High            0
Signal_Line_High     0
MACD_Low             0
Signal_Line_Low      0
MACD_Close           0
Signal_Line_Close    0
OBV_Open             0
OBV_High             0
OBV_Low              0
OBV_Close            0
%K_Open              0
%D_Open              0
%K_High              0
%D_High              0
%K_Low               0
%D_Low               0
%K_Close             0
%D_Close             0
Open_Percentage      0
High_Percentage      0
Low_Percentage       0
Volume_Percentage    0
Close_Percentage     0
dtype: int6

In [ ]:
t = int(0.8 * len(df1))
t

4641

In [ ]:
train_data = df1.iloc[ :t, : ]
test_data = df1.iloc[t: , :]

In [ ]:
train_mean = train_data.mean()
train_std = train_data.std()

In [ ]:
train_data = (train_data - train_mean) / train_std

In [ ]:
test_data = (test_data - train_mean) / train_std

In [ ]:
train_data = np.array(train_data)
test_data = np.array(test_data)

In [ ]:
X_train = train_data[:, :-1]
X_test = test_data[:, :-1]
y_train = train_data[:, -1]
y_test = test_data[:, -1]

In [ ]:
def analysis_data(X_train, X_test, y_train, y_test):
  result = []
  model = {
      'Random_Forest_Regressor': RandomForestRegressor(max_depth=50, n_estimators=500, random_state=42),
      'Adaboost_Regressor': AdaBoostRegressor(n_estimators=10, random_state=42),
      'Linear_Regressor': LinearRegression(normalize=True),
      'Decision_Tree_Regressor': DecisionTreeRegressor(criterion='friedman_mse'),
      'GradientBoost_Regressor': GradientBoostingRegressor(criterion='squared_error', random_state=42),
      'Ridge_Regressor': Ridge(alpha=1, fit_intercept=False, solver='svd'),
      'SGD_Regressor': SGDRegressor(),
      'Support_Vector_Regressor': SVR(C=10, gamma='auto', kernel='linear'),
      'The_ilsen_Regressor': TheilSenRegressor(),
      'Huber_Regressor': HuberRegressor(),
      'RANSAC_Regressor': RANSACRegressor(),
      'Hist_Gradient_Boosting_Regressor': HistGradientBoostingRegressor()
  }
  for key, value in model.items():
    value.fit(X_train, y_train)
    y_pred = value.predict(X_test)
    result.append({
        'Model_Name': key,
        'Training_Accuracy': value.score(X_train, y_train),
        'Testing_Accuracy': value.score(X_test, y_test),
        'R_Squared_Value (R2_Score)': r2_score(y_test, y_pred),
        'Mean_Absolute_Error (MAE)': metrics.mean_absolute_error(y_test, y_pred),
        'Mean_Squared_Error (MSE)' : metrics.mean_squared_error(y_test, y_pred),
        'Root_Mean_Squared_Error (RMSE)': np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
        'Mean_Absolute_Percentage_Error (MAPE)': metrics.mean_absolute_percentage_error(y_test, y_pred)
    })
  best_model = pd.DataFrame(result)
  Bestest_Model = best_model.sort_values(['R_Squared_Value (R2_Score)'],ignore_index=True, ascending=False)
  return Bestest_Model.style.hide_index()
analysis_data(X_train, X_test, y_train, y_test)

Model_Name,Training_Accuracy,Testing_Accuracy,R_Squared_Value (R2_Score),Mean_Absolute_Error (MAE),Mean_Squared_Error (MSE),Root_Mean_Squared_Error (RMSE),Mean_Absolute_Percentage_Error (MAPE)
Ridge_Regressor,0.967302,0.959917,0.959917,0.168099,0.052951,0.230111,23.364016
Random_Forest_Regressor,0.994082,0.957497,0.957497,0.132100,0.056148,0.236957,2.292606
Huber_Regressor,0.963257,0.953225,0.953225,0.177233,0.061792,0.248580,15.603583
SGD_Regressor,0.953105,0.931020,0.931020,0.247230,0.091125,0.301870,13.311124
Hist_Gradient_Boosting_Regressor,0.989923,0.927962,0.927962,0.158266,0.095166,0.308490,14.832151
GradientBoost_Regressor,1.000000,0.927025,0.927025,0.186235,0.096403,0.310488,1.059889
Decision_Tree_Regressor,1.000000,0.912768,0.912768,0.196318,0.115237,0.339466,0.852065
RANSAC_Regressor,0.976674,0.867835,0.867835,0.298353,0.174596,0.417847,14.127358
Linear_Regressor,0.979474,0.845278,0.845278,0.320056,0.204395,0.452100,30.793446
Adaboost_Regressor,0.801909,0.837880,0.837880,0.385177,0.214168,0.462783,46.233618


In [135]:
from sklearn.model_selection import TimeSeriesSplit

In [138]:
window = 365
n_splits = (df1.iloc[:t].shape[0] - window) // window
tscv = TimeSeriesSplit(n_splits=n_splits)
tscv

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=11, test_size=None)

In [139]:
best_estimators = {}
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'random_forest': {
        'model': RandomForestRegressor(random_state=42),
        'params' : {
            'n_estimators': [1,10,100,300,500],
            'criterion': ['squared_error','mse'],
            'max_depth': [5,10,20,30,50]
        }
    },
        'svm': {
        'model': svm.SVR(gamma='auto'),
        'params' : {
            'C': [1,10,20,40,60,80,100],
            'kernel': ['rbf','linear']
        }  
    },
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'ridge': {
            'model': Ridge(),
            'params': {
                'alpha' : [1, 10,100],
                'fit_intercept': [True, False], 
                'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
            }
        },
        'adaboost': {
        'model': AdaBoostRegressor(),
        'params': {
            'random_state' : [42],
            'n_estimators' : [1,10,100]
        }
        },
        'gradientboost': {
        'model': GradientBoostingRegressor(),
        'params': {
            'random_state' : [42],
            'n_estimators' : [1,10,100],
            'criterion' : ['squared_error','friedman_mse']
        }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=tscv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
        best_estimators[algo_name] = gs.best_estimator_

    global best_model_performance
    best_model_performance = pd.DataFrame(scores,columns=['model','best_score','best_params'])
    return best_model_performance

find_best_model_using_gridsearchcv(X_train, y_train)

,model,best_score,best_params
0,random_forest,0.951516,"{'criterion': 'squared_error', 'max_depth': 50..."
1,svm,0.962034,"{'C': 10, 'kernel': 'linear'}"
2,linear_regression,0.949579,{'normalize': True}
3,lasso,0.147257,"{'alpha': 1, 'selection': 'cyclic'}"
4,decision_tree,0.908127,"{'criterion': 'friedman_mse', 'splitter': 'best'}"
5,ridge,0.954029,"{'alpha': 1, 'fit_intercept': False, 'solver':..."
6,adaboost,0.814615,"{'n_estimators': 10, 'random_state': 42}"
7,gradientboost,0.925396,"{'criterion': 'squared_error', 'n_estimators':..."


In [142]:
best_estimators

{'random_forest': RandomForestRegressor(max_depth=50, n_estimators=500, random_state=42),
 'svm': SVR(C=10, gamma='auto', kernel='linear'),
 'linear_regression': LinearRegression(normalize=True),
 'lasso': Lasso(alpha=1),
 'decision_tree': DecisionTreeRegressor(criterion='friedman_mse'),
 'ridge': Ridge(alpha=1, fit_intercept=False, solver='svd'),
 'adaboost': AdaBoostRegressor(n_estimators=10, random_state=42),
 'gradientboost': GradientBoostingRegressor(criterion='squared_error', random_state=42)}

In [140]:
def best_model_performance_result(x,X_train,X_test,y_train,y_test):
  evaluation_metrics = []
  for i in range(len(x)):
    y_predicted = best_estimators[x['model'][i]].predict(X_test)
    evaluation_metrics.append({
        'Model': x['model'][i],
        'Train_Accuracy': best_estimators[x['model'][i]].score(X_train, y_train),
        'Validation_Accuracy': x['best_score'][i],
        'Test_Accuracy': best_estimators[x['model'][i]].score(X_test, y_test),
        'R2_Score': r2_score(y_test, y_predicted),
        'Mean_Absolute_Error': metrics.mean_absolute_error(y_test, y_predicted),
        'Mean_Squared_Error': metrics.mean_squared_error(y_test, y_predicted),
        'Root_Mean_Squared_Error': np.sqrt(metrics.mean_squared_error(y_test, y_predicted)),
        'Mean_Absolute_Percentage_Error': metrics.mean_absolute_percentage_error(y_test, y_predicted)
        }
    )
  Best_Model_Performance_Analysis_Result = pd.DataFrame(evaluation_metrics, columns=['Model','Train_Accuracy','Validation_Accuracy','Test_Accuracy','R2_Score','Mean_Absolute_Error','Mean_Squared_Error','Root_Mean_Squared_Error','Mean_Absolute_Percentage_Error'], index=None)
  Bestest_Model = Best_Model_Performance_Analysis_Result.sort_values(['R2_Score'],ignore_index=True, ascending=False)
  return Bestest_Model.style.hide_index()
best_model_performance_result(best_model_performance,X_train,X_test,y_train,y_test)

Model,Train_Accuracy,Validation_Accuracy,Test_Accuracy,R2_Score,Mean_Absolute_Error,Mean_Squared_Error,Root_Mean_Squared_Error,Mean_Absolute_Percentage_Error
ridge,0.967302,0.954029,0.959917,0.959917,0.168099,0.052951,0.230111,23.364016
random_forest,0.994239,0.951516,0.959273,0.959273,0.130261,0.053802,0.231952,1.521862
gradientboost,0.981063,0.925396,0.942474,0.942474,0.160232,0.075995,0.275672,1.715495
svm,0.974894,0.962034,0.925869,0.925869,0.228592,0.097931,0.312939,36.803024
decision_tree,1.000000,0.908127,0.914923,0.914923,0.193078,0.112391,0.335247,0.821140
adaboost,0.890104,0.814615,0.900484,0.900484,0.258223,0.131465,0.362582,22.768331
linear_regression,0.979474,0.949579,0.845278,0.845278,0.320056,0.204395,0.452100,30.793446
lasso,0.000000,0.147257,-0.001607,-0.001607,0.805508,1.323169,1.150291,1.000000


In [147]:
Ridge().get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'positive', 'random_state', 'solver', 'tol'])

In [148]:
ridge = Ridge()
ridge.fit(X_train, y_train)

Ridge()

In [149]:
ridge.score(X_train, y_train)

0.9673015378927821

In [150]:
ridge.score(X_test, y_test)

0.9599173333306503